<a href="https://colab.research.google.com/github/Elheity/Dr_Nadeem/blob/main/comp_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load
import os
import itertools
import sklearn.metrics

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

from skimage import io
from skimage.transform import resize

import tensorflow as tf
from tensorflow import keras
import tensorflow as tf
from tensorboard.plugins.hparams import api as hp
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.preprocessing import image

import matplotlib.pyplot as plt
from skimage.color import gray2rgb
import matplotlib.pyplot as plt
# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# New Section

In [3]:
paths = []
labels = []
def create_paths_labels(path):
    for root,dirs,files in os.walk(path):
        for pics in files:
            #print(os.path.join(root, pics))
            paths.append(os.path.join(root, pics))
            if os.path.basename(root) == 1:
                labels.append(1)
            elif os.path.basename(root) == 0:
                labels.append(0)
    return [paths,labels]

In [3]:
paths = []
labels = []
path = r"/content/drive/MyDrive/train"
for root,dirs,files in os.walk(path):
        for pics in files:
            #print(os.path.join(root, pics))
            paths.append(os.path.join(root, pics))
            #print( 'x',dirs)
            if str(os.path.basename(root)) == '1':
                labels.append(1)
                #print( 'y',root)
            elif str(os.path.basename(root)) == '0':
                labels.append(0)

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
len(paths)
paths = paths[:-1]
paths[-1]

'/content/drive/MyDrive/train/1/41584_1735459739.png'

In [ ]:
i = 0
imgs = []
for imgph in paths:
    img = io.imread(imgph)
    i += 1
    img_resized = resize(img ,(128,128))
    imgs.append(img_resized)


In [7]:
print(len(imgs))

11449


In [ ]:
x =[]
# Create a data generator object
datagen = ImageDataGenerator(
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest')

# Load an image from file
for i in paths:
  img = image.load_img(i, target_size=(150, 150))
  # Convert the image to a numpy array
  x = image.img_to_array(img)
  # Reshape the array to add a dimension for the batch size
  x = x.reshape((1,) + x.shape)

  # Generate and save augmented images
  i = 0
  for batch in datagen.flow(x, batch_size=1, save_to_dir='/content/preview', save_prefix='img', save_format='jpeg'):
      i += 1
      if i > 20:
          break

In [8]:
def convert_path_toImage(paths):
    imgs = []
    for path in paths:
        img = io.imread(path)
        img = resize(img , 128,128)
        
    return imgs

In [ ]:
#train_imgs = create_paths_labels(r"/kaggle/input/mammography-breast-cancer-detection/train")
#train_imgs = convert_path_toImage(train_imgs)

In [1]:
len(x)

NameError: ignored

In [10]:
train_imgs = np.array(imgs)
train_labels = np.array(labels)

In [11]:
train_labels = train_labels[:-1]

In [12]:
print(len(train_imgs))
print(len(train_imgs.shape))
print(len(train_labels))
train_imgs.shape

11449
3
11449


(11449, 128, 128)

In [13]:
X_valid, X_train = train_imgs[:1500] / 255.0, train_imgs[1500:] / 255.0
y_valid, y_train = train_labels[:1500], train_labels[1500:]

In [14]:
X_train.shape

(9949, 128, 128)

In [15]:
# Defining the hypermatarest we would test and their range
HP_DROPOUT = hp.HParam('dropout', hp.Discrete([ 0.3, 0.4]))
# Defining the hyperparameters we would tune, and their values to be tested
HP_FILTER_SIZE = hp.HParam('filter_size', hp.Discrete([3 , 5]))
HP_FILTER_NUM = hp.HParam('filters_number', hp.Discrete([64, 128,192]))
METRIC_ACCURACY = 'accuracy'

# Logging setup info
with tf.summary.create_file_writer(r'Logs/Model 1/hparam_tuning/').as_default():
    hp.hparams_config(
        hparams=[HP_DROPOUT],
        metrics=[hp.Metric(METRIC_ACCURACY, display_name='Accuracy')],
    )

In [16]:
EPOCHS = 15
BATCH_SIZE = 40

In [2]:
# define the generator
train_gen = ImageDataGenerator()

NameError: ignored

In [17]:
# Wrapping our model and training in a function
def train_test_model(hparams, session_num):
    global model
    # Outlining the model/architecture of our CNN
    model = tf.keras.Sequential([
        tf.keras.layers.Conv2D(hparams[HP_FILTER_NUM], hparams[HP_FILTER_SIZE],
                               activation='relu',
                               padding="same",
                               input_shape=(128,128,1)),
        tf.keras.layers.MaxPooling2D(2),
        tf.keras.layers.Conv2D(hparams[HP_FILTER_NUM], 3, padding="same", activation='relu'),
        tf.keras.layers.MaxPooling2D(2),
        #tf.keras.layers.Conv2D(hparams[HP_FILTER_NUM], 3, padding="same", activation='relu'),
        #tf.keras.layers.MaxPooling2D(2),
        tf.keras.layers.Flatten(),
        #tf.keras.layers.Dense(1024, activation='relu'),
        #tf.keras.layers.Dropout(hparams[HP_DROPOUT]),
        tf.keras.layers.Dense(256, activation='relu'),
        tf.keras.layers.Dropout(hparams[HP_DROPOUT]),
        tf.keras.layers.Dense(128, activation='relu'),
        tf.keras.layers.Dropout(hparams[HP_DROPOUT]),
        tf.keras.layers.Dense(2, activation="softmax")
    ])
    #model.summary()
    # Defining the loss function
    loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
    # use generator to train
    model.fit_generator(train_gen.flow(X_train, y_train, batch_size=32),
                        steps_per_epoch=len(X_train) / 32, epochs=10)
    # Compiling the model with parameter value for the optimizer
    model.compile(optimizer='adam', loss=loss_fn, metrics=['accuracy'])
    

    
    
    def plot_confusion_matrix(cm, class_names):
        """
        Returns a matplotlib figure containing the plotted confusion matrix.

        Args:
          cm (array, shape = [n, n]): a confusion matrix of integer classes
          class_names (array, shape = [n]): String names of the integer classes
        """
        figure = plt.figure(figsize=(8, 8))
        plt.imshow(cm, interpolation='nearest', cmap=plt.cm.Blues)
        plt.title("Confusion matrix")
        plt.colorbar()
        tick_marks = np.arange(len(class_names))
        plt.xticks(tick_marks, class_names, rotation=45)
        plt.yticks(tick_marks, class_names)

        # Normalize the confusion matrix.
        cm = np.around(cm.astype('float') / cm.sum(axis=1)[:, np.newaxis], decimals=2)

        # Use white text if squares are dark; otherwise black.
        threshold = cm.max() / 2.
        for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
            color = "white" if cm[i, j] > threshold else "black"
            plt.text(j, i, cm[i, j], horizontalalignment="center", color=color)

        plt.tight_layout()
        plt.ylabel('True label')
        plt.xlabel('Predicted label')
        return figure
    
    
    # Logging the training process data to use later in tensorboard
    log_dir = "Logs\\Model 6\\fit\\" + "run-{}".format(session_num)
    tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1, profile_batch=0)
    file_writer_cm = tf.summary.create_file_writer(log_dir + '/cm')  
    
    
    def plot_to_image(figure):
        """Converts the matplotlib plot specified by 'figure' to a PNG image and
        returns it. The supplied figure is closed and inaccessible after this call."""
        # Save the plot to a PNG in memory.
        buf = io.BytesIO()
        plt.savefig(buf, format='png')
        # Closing the figure prevents it from being displayed directly inside
        # the notebook.
        plt.close(figure)
        buf.seek(0)
        # Convert PNG buffer to TF image
        image = tf.image.decode_png(buf.getvalue(), channels=4)
        # Add the batch dimension
        image = tf.expand_dims(image, 0)
        return image
    
    def log_confusion_matrix(epoch, logs):
        # Use the model to predict the values from the validation dataset.
        test_pred_raw = model.predict(X_valid)
        test_pred = np.argmax(test_pred_raw, axis=1)

        # Calculate the confusion matrix.
        cm = sklearn.metrics.confusion_matrix(y_valid, test_pred)
        # Log the confusion matrix as an image summary.
        figure = plot_confusion_matrix(cm, class_names = ["yes", "no"])
        cm_image = plot_to_image(figure)

        # Log the confusion matrix as an image summary.
        with file_writer_cm.as_default():
            tf.summary.image("Confusion Matrix", cm_image, step=epoch)
    
    
    # Define the per-epoch callback.
    cm_callback = tf.keras.callbacks.LambdaCallback(on_epoch_end=log_confusion_matrix)

    
    # Defining early stopping to prevent overfitting
    early_stopping = tf.keras.callbacks.EarlyStopping(
        monitor = 'val_loss',
        mode = 'auto',
        min_delta = 0,
        patience = 2,
        verbose = 0, 
        restore_best_weights = True
    )
    
    # Training the model
    model.fit(
        X_train,
        y_train,
        epochs = EPOCHS,
        batch_size = BATCH_SIZE,
        callbacks = [tensorboard_callback, cm_callback, early_stopping],
        validation_data = (X_valid,y_valid),
        verbose = 2
    )
    
    _, accuracy = model.evaluate(X_valid,y_valid)
    
    model.save(r"saved_models\Model 1\Run-{}".format(session_num))
    
    

    return accuracy

In [ ]:
test_loss, test_accuracy = model.evaluate(images_test, y_test)
print('////////////////////////////////// Testing Part /////////////////////////////////////')
print('////////////////////////////////////////////////////////////////////////////////////')
print('Test loss: {0:.4f}. Test accuracy: {1:.2f}%'.format(test_loss, test_accuracy*100.))

In [18]:
# Creating a function to log the resuls
def run(log_dir, hparams, session_num):
    
    with tf.summary.create_file_writer(log_dir).as_default():
        hp.hparams(hparams)  # record the values used in this trial
        accuracy = train_test_model(hparams, session_num)
        tf.summary.scalar(METRIC_ACCURACY, accuracy, step=1)

In [1]:
session_num = 1

for filter_size in HP_FILTER_SIZE.domain.values:
    for filter_num in HP_FILTER_NUM.domain.values:
        for dropout in HP_DROPOUT.domain.values:

            hparams = {
                HP_FILTER_SIZE: filter_size,
                HP_FILTER_NUM: filter_num,
                HP_DROPOUT: dropout
            }

            run_name = "run-%d" % session_num
            print('--- Starting trial: %s' % run_name)
            print({h.name: hparams[h] for h in hparams})
            run('Logs/Model 1/hparam_tuning/' + run_name, hparams, session_num)

            session_num += 1


NameError: ignored

In [ ]:
# Creating a function to log the resuls
def run(log_dir, hparams, session_num):
    
    with tf.summary.create_file_writer(log_dir).as_default():
        hp.hparams(hparams)  # record the values used in this trial
        accuracy = train_test_model(hparams, session_num)
        tf.summary.scalar(METRIC_ACCURACY, accuracy, step=1)

In [ ]:
from keras.layers import Input, Add, Dense, Activation, ZeroPadding2D, BatchNormalization, Flatten, Conv2D
from keras.layers import AveragePooling2D, MaxPooling2D, Dropout
from keras.models import Model

def identity_block(X, f, filters, stage, block):
    """
    Implementation of the identity block
    """
    # defining name basis
    conv_name_base = 'res' + str(stage) + block + '_branch'
    bn_name_base = 'bn' + str(stage) + block + '_branch'
    
    # Retrieve Filters
    F1, F2, F3 = filters
    
    # Save the input value. You'll need this later to add back to the main path. 
    X_shortcut = X
    
    # First component of main path
    X = Conv2D(filters = F1, kernel_size = (1, 1), strides = (1,1), padding = 'valid', name = conv_name_base + '2a', kernel_initializer = glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis = 3, name = bn_name_base + '2a')(X)
    X = Activation('relu')(X)
    
    # Second component of main path
    X = Conv2D(filters = F2, kernel_size = (f, f), strides = (1,1), padding = 'same', name = conv_name_base + '2b', kernel_initializer = glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis = 3, name = bn_name_base + '2b')(X)
    X = Activation('relu')(X)

    # Third component of main path
    X = Conv2D(filters = F3, kernel_size = (1, 1), strides = (1,1), padding = 'valid', name = conv_name_base + '2c', kernel_initializer = glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis = 3, name = bn_name_base + '2c')(X)

    # Final step: Add shortcut value to main path, and pass it through a RELU activation
    X = Add()([X, X_shortcut])
    X = Activation('relu')(X)
    
    return X


